In [1]:
from ckiptagger import construct_dictionary, WS, POS
ws = WS("C:/Users/jason/data")
pos = POS("C:/Users/jason/data")

#載入自定義字典
dictionary1 = {}
with open("word_to_weight.txt", encoding = "utf-8") as f:
    for line in f:
       (key, val) = line.split()
       dictionary1[key] = int(val)

#斷詞、詞性標註結果整理        
def w_pos(word):
    a_ws = ws([word], coerce_dictionary = construct_dictionary(dictionary1))
    a_pos = pos(a_ws)
    b = " ".join('%s' % a_pos[0][id] for id in range(len(a_pos[0])))
    c = list(zip(a_ws[0], a_pos[0]))
    print(c)
    return c

In [2]:
text = input("請輸入文字：")

請輸入文字：我想知道銷售人員的銷售額


In [3]:
a_w_pos = w_pos(text)
a_list_ws, a_list_pos = [], []
for i in range(len(a_w_pos)):
    a_list_ws += [a_w_pos[i][0]]
    #詞性
    a_list_pos += [a_w_pos[i][1]]

[('我', 'Nh'), ('想', 'VE'), ('知道', 'VK'), ('銷售人員', 'Na'), ('的', 'DE'), ('銷售額', 'Na')]


In [4]:
kw = {"time":'', "location":'', "VH":'', "chart":''}
cnt_kw_dict = 1

for i in range(len(a_list_pos)):
    try:
        if a_list_pos[i] == 'Nd':
            kw['time'] += a_list_ws[i]
        elif a_list_pos[i] == 'Nc':
            kw['location'] += a_list_ws[i]
        elif a_list_pos[i] == 'VH':
            kw['VH'] += a_list_ws[i]
        elif a_list_pos[i] == 'Caa':
            kw[cnt_kw_dict] = a_list_ws[i-1]
            cnt_kw_dict += 1
        elif a_list_pos[i] == 'DE':
            kw[cnt_kw_dict] = a_list_ws[i-1]
            kw[cnt_kw_dict + 1] = a_list_ws[i+1]
            cnt_kw_dict += 2
        elif a_list_pos[i][:2] != 'Ne' and a_list_pos[i] != 'Nf' and a_list_pos[i][0] == 'N' and a_list_pos[i+1][0] == 'N' :
            kw[cnt_kw_dict] = a_list_ws[i]
            kw[cnt_kw_dict + 1] = a_list_ws[i+1]
            cnt_kw_dict += 2
    except:
        pass
if a_list_pos[1] == 'Na':
    kw['chart'] += a_list_ws[1]
if cnt_kw_dict >= 3 and kw['chart'][1:] == '條圖':
    kw['chart'] = "堆疊" + kw['chart']
    
print(kw)
import json

dict_json = json.dumps(kw, ensure_ascii=False)
with open('kw.json', 'w', encoding = "utf-8") as file:
    file.write(dict_json)

{'time': '', 'location': '', 'VH': '', 'chart': '', 1: '銷售人員', 2: '銷售額'}


In [5]:
def sim(kw):
    zz = kw * 5
    # 讀入以空格插入於字與字中的文件
    N_dict_list=[]
    with open('dict_list1.txt', 'r') as file:
        for i in file:
            N_dict_list.append(i.strip('\n').strip("['").strip("']").split("', '"))
    N_dict_list[0].insert(0, ' '.join(i for i in zz))
    #字詞相似性
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.metrics.pairwise import cosine_distances
    vectorizer = CountVectorizer(token_pattern = r"(?u)\b\w+\b")
    X = vectorizer.fit_transform(N_dict_list[0])
    csd = cosine_distances(X)
    a = csd[0,:].argsort()[1]
    #原始欄位名稱
    colName = []
    cnt = 0
    with open('dict_list.txt', 'r') as file:
        for i in file:
            colName.append(i.strip('\n').strip("['").strip("']").split("', '"))
    if csd[0, a] > 0.99 :
        r = "N N,"+ kw
        return r
    else :
        r = colName[0][a-1]
        rs = [colName[0][a-1]]
        rs_S = [str(1-csd[0, a])]
        for i in range(1, len(csd)-1):
            if abs(csd[0, a] - csd[0, csd[0,:].argsort()[i+1]])<0.01:
                rs.append(colName[0][csd[0,:].argsort()[i+1]-1])
                rs_S.append(str(1-csd[0, csd[0,:].argsort()[i+1]]))
            else:
                break
        if len(rs)>1:
            print(rs,rs_S)
            return rs
        else:
            print(r,1-csd[0, a])
            return r

In [6]:
def sep_text(txtfile):
    d = {}
    with open(txtfile , encoding = "utf-8") as f:
        for line in f:
            (key, val) = line.strip('\n').split('$')
            d[key] = val
    for k in d:
        d[k] = d[k].split(",")
    dict_list = []
    # i + j 把表格名稱與欄位連在一起
    #append 加入在list 中
    for i in d:
        #a += i
        for j in d[i]:
             dict_list.append(i + ',' +j)
    with open('dict_list.txt', 'w') as file:
        file.write(str(dict_list))
    #join 將文字分割
    dict_list_s = [' '] * len(dict_list)
    for j in range(len(dict_list)):
        dict_list_s[j] = ' '.join(i for i in dict_list[j])
    with open('dict_list1.txt', 'w') as file:
        file.write(str(dict_list_s))
    return

sep_text("dataName.txt")

In [7]:
import json

with open('kw.json' , 'r', encoding = "utf-8") as file:
    jf = json.loads(file.read())

pos = ["多", "高", "大", "好", "佳", "優"]
neg = ["少", "低", "小", "壞", "差", "劣"]
for i in jf:
    if i == 'VH':
        for j in pos:
            if j in jf[i]:
                jf[i] = "DESC"
        for j in neg:
            if j in jf[i]:
                jf[i] = "ASC"
    else:
        jf[i] = sim(jf[i])
        
with open('kw_N.json' , 'w', encoding = "utf-8") as file:
    json.dump(jf, file, ensure_ascii = False)

銷售 人員資料,銷售人員編號 0.8944271909999159
['銷售 銷售訂單明細,產品銷售額', '銷售 人員資料,銷售額', '銷售 地區資料,銷售額'] ['0.808290376865476', '0.8006407690254357', '0.8006407690254357']


In [8]:
del pos
pos = POS("C:/Users/jason/data")